In [ ]:
import gc
from all_stand_var import conv_dict,used_cols
from all_own_funct import memory_downscale,memory_upscale,evaluate,lin_reg_coef
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix
from matplotlib.backends.backend_pdf import PdfPages
import pickle
import locale
import LR_build_CHD as pp
import datetime as dt

from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

locale.setlocale(locale.LC_ALL, 'fr_FR')

# Output folder
output_folder = os.path.join(os.getcwd(), 'Results_LR_RF_CHD_v3','Static_redo')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


In [ ]:
import tables

# read in data
df=pd.read_hdf(os.path.join('./Results_CHD_v5/final', 'processed_df.h5'),key='df',mode='r')
#df=memory_downscale(df)
print(df.info())




In [ ]:
# Descriptive statistics of all used columns
#used_cols = [ 'pat_datetime',  'mon_rr', 'mon_hr', 'mon_sat',
#              'mon_etco2', 'vent_m_fio2', 'vent_m_ppeak','vent_m_peep',
#             'mon_ibp_mean','pat_weight_act','vent_m_rr', 'vent_m_tv_exp','Age']
#for col in used_cols:
#    print(df[col].describe())

In [ ]:
# Figures of different patient trajectories
"""
print(df[df.index.get_level_values('pat_hosp_id') == 8460454]['Reintub'].value_counts())
print(df[df.index.get_level_values('pat_hosp_id') == 9852266]['pat_datetime'].min())
print(df[df.index.get_level_values('pat_hosp_id') == 9852266]['pat_datetime'].max())
plt.plot(np.linspace(start=0,stop=720,num=720),df[df.index.get_level_values('pat_hosp_id') == 3204962]['vent_m_peep'][:720],'r-',label='Failed extubation')
plt.plot(np.linspace(start=0,stop=720,num=720),df[df.index.get_level_values('pat_hosp_id') == 9852266]['vent_m_peep'],'b-',label='Successful extubation')
plt.legend(loc=1)
plt.xlabel('Time (min)')
plt.ylabel('Scaled value for positive end expiratory pressure (PEEP)')
plt.tight_layout()
fig=plt.gcf()
plt.savefig(os.path.join(output_folder,'peep.png'), format='png',
             dpi=300, facecolor='white', transparent=True, bbox_inches='tight')
fig.set_size_inches(12, 8)
plt.show()




plt.plot(np.linspace(start=0,stop=720,num=720),df[df.index.get_level_values('pat_hosp_id') == 1497966]['mon_etco2'],'r-',label='Failed extubation')
plt.plot(np.linspace(start=0,stop=720,num=720),df[df.index.get_level_values('pat_hosp_id') == 7795892]['mon_etco2'],'b-',label='Successful extubation')
plt.xlabel('Time (min)')
plt.ylabel('Scaled value for end-tdail CO2 ')
plt.legend(loc=1)
fig=plt.gcf()
fig.set_size_inches(12, 8)
plt.savefig(os.path.join(output_folder,'etco2.png'), format='png',
             dpi=300, facecolor='white', transparent=True, bbox_inches='tight')
plt.show()



plt.plot(np.linspace(start=0,stop=720,num=720),df[df.index.get_level_values('pat_hosp_id') == 5659148]['mon_ibp_mean'],'r-',label='Failed extubation')
plt.plot(np.linspace(start=0,stop=720,num=720),df[df.index.get_level_values('pat_hosp_id') == 7795892]['mon_ibp_mean'],'b-',label='Successful extubation')
plt.xlabel('Time (min)')
plt.ylabel('Scaled value for mean invasive blooppressure')
plt.legend(loc=1)
fig=plt.gcf()
fig.set_size_inches(12, 8)
plt.savefig(os.path.join(output_folder,'bp.png'), format='png',
             dpi=300, facecolor='white', transparent=True, bbox_inches='tight')
"""

In [ ]:
import datetime as dt
from sklearn.preprocessing import StandardScaler

df=df.reset_index(drop=False)
# calculate missingness
def index_1hr(group):
    #grouped = pd.Grouper(key='pat_datetime', freq='1H')
    #group['idx_1hr']=group.groupby(grouped,sort=False).ngroup().add(1)
    #group['idx_1hr']=group['idx_1hr'].apply(str)
    group['mis'] = group['pat_datetime'].max()-group['pat_datetime'].min()
    group['mis'] = group['mis'].dt.seconds.divide(60)
    return group
#df.reset_index(drop=True,inplace=True)
df=df.groupby('Adnum',sort=False,as_index=False).apply(index_1hr)
# Scale missingness
df[['mis']]=StandardScaler().fit_transform(df[['mis']])
df=df.set_index(['pat_hosp_id','OK_datum'])


In [ ]:
# Stratified split to train, test and validation set
pat=pd.read_excel(r'Results_CHD\admissiondate_CHD0.xlsx',
                    parse_dates=['OK_datum'],index_col=0)
group=pat.groupby('pat_hosp_id',sort=False).max().reset_index()
group.drop_duplicates('pat_hosp_id',inplace=True)

df_train,df_val,df_test=pp.split_stratified_into_train_val_test(group, stratify_colname='Reintub',
                                         frac_train=0.7, frac_val=0.1, frac_test=0.2,
                                         random_state=1)

train_pat=df_train['pat_hosp_id'].unique()
test_pat=df_test['pat_hosp_id'].unique()
val_pat=df_val['pat_hosp_id'].unique()

df_train = df[df.index.get_level_values('pat_hosp_id').isin(train_pat)]
df_val = df[df.index.get_level_values('pat_hosp_id').isin(val_pat)]
df_test = df[df.index.get_level_values('pat_hosp_id').isin(test_pat)]

In [ ]:
# from dataframwe with label per minute, to array with label per admission
def y_modelling(df):
    y=df[['Reintub','Adnum']]
    y=y.groupby(['Adnum'],sort=False).agg(['max'])
    y.columns = ["_".join(a) for a in y.columns.to_flat_index()]
    y=y.reset_index().drop_duplicates().set_index('Adnum')
    y=y[~y.index.duplicated(keep='last')]
    return y
Y_TRAIN=y_modelling(df_train)
Y_TEST=y_modelling(df_test)
Y_VAL=y_modelling(df_val)
Y_TRAIN=Y_TRAIN['Reintub_max'].to_numpy()
Y_TEST=Y_TEST['Reintub_max'].to_numpy()
Y_VAL=Y_VAL['Reintub_max'].to_numpy()
Y_TRAIN=np.append(Y_TRAIN,Y_VAL)


In [ ]:
def x_modelling(df):
    df=df[['Adnum','mis','pat_weight_act','Age','Diagnose']]
    #df=df.drop(['Extubation_date','level_0','pat_datetime', 'Reintub'],axis=1)
    df=df.groupby(['Adnum'],sort=False).agg(['mean'])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df=df.reset_index().drop_duplicates().set_index('Adnum')
    return df
X_TRAIN=memory_upscale(x_modelling(df_train))
X_TEST=memory_upscale(x_modelling(df_test))
X_VAL=memory_upscale(x_modelling(df_val))

X_TRAIN=X_TRAIN.fillna(value=0)
X_VAL=X_VAL.fillna(value=0)
X_TEST=X_TEST.fillna(value=0)
X_TRAIN=X_TRAIN.append(X_VAL)

In [ ]:
# Save train and test data
f = open(os.path.join(output_folder, 'x_train.txt'), 'wb')
pickle.dump(X_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'x_test.txt'), 'wb')
pickle.dump(X_TEST, f)
f.close()

f = open(os.path.join(output_folder, 'y_train.txt'), 'wb')
pickle.dump(Y_TRAIN, f)
f.close()

f = open(os.path.join(output_folder, 'y_test.txt'), 'wb')
pickle.dump(Y_TEST, f)
f.close()

In [ ]:
# Optimizer set up for random forest
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_TRAIN, Y_TRAIN)




In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,average_precision_score,f1_score,roc_curve,roc_auc_score,plot_confusion_matrix

# Evaluate the best and base model
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42,max_depth=10)
base_model.fit(X_TRAIN, Y_TRAIN)
base_accuracy = evaluate(base_model, X_TEST, Y_TEST,'base_accuracy RF',output_folder)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_TEST,Y_TEST,'Best_random RF',output_folder)
if base_accuracy > random_accuracy:
    best_random=base_model


In [ ]:
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from matplotlib import cm
# calculate Brier score loss, not used
"""
# Gaussian Naive-Bayes with no calibration
# GaussianNB itself does not support sample-weights
prob_pos_clf = best_random.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(best_random, cv=2, method='isotonic')
clf_isotonic.fit(X_TRAIN, Y_TRAIN)
prob_pos_isotonic = clf_isotonic.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(best_random, cv=2, method='sigmoid')
clf_sigmoid.fit(X_TRAIN, Y_TRAIN)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X_TEST)[:, 1]

print("Brier score losses: (the smaller the better)")

clf_score = brier_score_loss(Y_TEST, prob_pos_clf)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(Y_TEST, prob_pos_isotonic)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(Y_TEST, prob_pos_sigmoid)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for RF on training set\n\n")
    file.write("Brier score losses: (the smaller the better)\n")
    file.write("No calibration: %1.3f" % clf_score)
    file.write("With isotonic calibration: %1.3f" % clf_isotonic_score)
    file.write("With sigmoid calibration: %1.3f" % clf_sigmoid_score)
"""



In [ ]:
# Create AUROC plot, confusion matrix and other results for Random Forest

from all_own_funct import roc_auc_ci
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures_alls.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures_all.pdf"))
    os.remove(os.path.join(output_folder,f"Result_scores_all.txt"))
clf = best_random 


y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

# write scoring metrics to file
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for RF on training set\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of random forrest")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)

n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write("\nConfidence interval for the score: [{:0.3f} - {:0.3}]\n".format(
    confidence_lower, confidence_upper))
a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)


plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)

plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Random Forrest hour data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)



In [ ]:
# Hyperparameters for Logistic Regression
# The solvers
solver =['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# Class weight optimizer
class_weight=[{0:0.1,1:0.9},'balanced',None]
# Penalty
penalty=['l1', 'l2', 'elasticnet', None]
# Inverse of regularization strength
C=np.logspace(-3,3,7)
# Bootsrapping
bootstrap = [True, False]
# Create the random grid
random_grid = {'solver':solver,
                'class_weight':class_weight,
                'penalty':penalty,
                'C':C,
                'penalty':penalty}

# First create the base model to tune
lr = LogisticRegression(max_iter=1000)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
lr_random = RandomizedSearchCV(estimator = lr , param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
lr_random.fit(X_TRAIN, Y_TRAIN)

In [ ]:
# Evaluate the base and optimized model
base_model_lr = LogisticRegression(max_iter=1000)
base_model_lr.fit(X_TRAIN, Y_TRAIN)
base_accuracy_lr = evaluate(base_model_lr, X_TEST, Y_TEST,'base LR',output_folder)

best_random_lr = lr_random.best_estimator_
random_accuracy_lr = evaluate(best_random_lr, X_TEST,Y_TEST,'Best Random LR',output_folder)

if base_accuracy_lr > random_accuracy_lr:
    best_random_lr=base_model_lr

# Brier scores
"""
# Gaussian Naive-Bayes with no calibration
# GaussianNB itself does not support sample-weights
prob_pos_clf = best_random_lr.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with isotonic calibration
clf_isotonic = CalibratedClassifierCV(best_random_lr, cv=2, method='isotonic')
clf_isotonic.fit(X_TRAIN, Y_TRAIN)
prob_pos_isotonic = clf_isotonic.predict_proba(X_TEST)[:, 1]

# Gaussian Naive-Bayes with sigmoid calibration
clf_sigmoid = CalibratedClassifierCV(best_random_lr, cv=2, method='sigmoid')
clf_sigmoid.fit(X_TRAIN, Y_TRAIN)
prob_pos_sigmoid = clf_sigmoid.predict_proba(X_TEST)[:, 1]

print("Brier score losses: (the smaller the better)")

clf_score = brier_score_loss(Y_TEST, prob_pos_clf)
print("No calibration: %1.3f" % clf_score)

clf_isotonic_score = brier_score_loss(Y_TEST, prob_pos_isotonic)
print("With isotonic calibration: %1.3f" % clf_isotonic_score)

clf_sigmoid_score = brier_score_loss(Y_TEST, prob_pos_sigmoid)
print("With sigmoid calibration: %1.3f" % clf_sigmoid_score)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for LR on training set\n\n")
    file.write("Brier score losses: (the smaller the better)\n")
    file.write("No calibration: %1.3f" % clf_score)
    file.write("With isotonic calibration: %1.3f" % clf_isotonic_score)
    file.write("With sigmoid calibration: %1.3f" % clf_sigmoid_score)
"""

In [ ]:
# Create AUROC plot, confusion matrix and other results for logistic regression

clf = best_random_lr


y_pred_clas=clf.predict(X_TEST)
# Predict the probabilities, function depends on used classifier

try:
    y_pred_prob=clf.predict_proba(X_TEST)
    y_pred_prob=y_pred_prob[:,1]
except:
    try:
        y_pred_prob=clf.decision_function(X_TEST)
    except:
        y_pred_prob=y_pred_clas

report=classification_report(Y_TEST,y_pred_clas,target_names=['No Reintubation','Reintubation'])
score=clf.score(X_TEST,Y_TEST)
average_precision = average_precision_score(Y_TEST, y_pred_prob)
f1_s=f1_score(Y_TEST, y_pred_clas)

with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write(f"Results for LR on training\n\n")
    file.write(f"Classification report \n {report} \n")
    file.write(f"Hold_out_scores {score} \n")
    file.write(f"Average precision score {average_precision} \n")
    file.write(f"F1 score {f1_s} \n\n\n")

plot_confusion_matrix(clf,X_TEST,Y_TEST)
plt.title(f"Confusion matrix of logistic regression")
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)

fpr, tpr, _ = roc_curve(Y_TEST,  y_pred_prob)
auc = roc_auc_score(Y_TEST, y_pred_prob)


n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_pred_prob), len(y_pred_prob))
    if len(np.unique(Y_TEST[indices])) < 2:
        # We need at least one positive and one negative sample for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(Y_TEST[indices], y_pred_prob[indices])
    bootstrapped_scores.append(score)
    #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

# Computing the lower and upper bound of the 90% confidence interval
# You can change the bounds percentiles to 0.025 and 0.975 to get
# a 95% confidence interval instead.
confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
with open(os.path.join(output_folder,f"Result_scores_all.txt"),'a') as file:
    file.write("\nConfidence interval for the score: [{:0.3f} - {:0.3}]\n".format(
    confidence_lower, confidence_upper))

a=roc_auc_ci(Y_TEST,y_pred_prob)
print(a)

plt.plot(fpr,tpr,label=f"auc={auc}",linewidth=1.5,markersize=1)


plt.legend(loc=4,fontsize='xx-small')
plt.title(f'ROC of Logistic Regression data')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)
pdf.close()


In [ ]:
# Save the best models
import pickle
f = open(os.path.join(output_folder,'ran_for.sav'), 'wb')
pickle.dump(best_random, f)
f.close()

f = open(os.path.join(output_folder,'log_reg.sav'), 'wb')
pickle.dump(best_random_lr, f)
f.close()